# APIM ❤️ OpenAI

## Private connectivity lab
![flow](../../images/backend-pool-load-balancing.gif)

Playground to show how to create a private network for consuming LLMs from `AI Services`.
This lab demonstrates how to create a private network for consuming LLMs from `AI Services` using `Private Link Services`, `Azure API Management (APIM)` and `Azure Front Door`.

Notes:
- **This is a typical prioritized PTU with fallback consumption scenario**. The lab specifically showcases how a priority 1 (highest) backend is exhausted before gracefully falling back to two equally-weighted priority 2 backends.
- The backend pool uses round-robin by default.
- Priority and weight-based routing are supported and can be adjusted by modifying `priority` (the lower the number, the higher the priority) and `weight` parameters in the `openai_resources` variable below.
- The `retry` API Management policy initiates a retry to an available backend if an HTTP 429 status code is encountered. This is transparent to the caller.

### Result
![result](result.png)

### Prerequisites
- [Python 3.12 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) and matplotlib installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [29]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"frc-demo-lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "francecentral"

apim_sku = 'Standardv2'

# Prioritize East US until exhaustion (simulate PTU with TPM), then equally distribute between Sweden and West US (consumption fallback)
openai_resources = [
    {"name": "openai1", "capacity": 20, "location": "eastus", "priority": 1},
    {"name": "openai2", "capacity": 20, "location": "swedencentral", "priority": 2, "weight": 50},
    {"name": "openai3", "capacity": 20, "location": "westus", "priority": 2, "weight": 50}
]

openai_deployment_name = "gpt-4o-mini"
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_capacity = 8
openai_model_sku = 'Standard'
openai_api_version = "2024-02-01"

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 17:09:54.215355 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [30]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 17:10:01.474803 :1s]
👉🏽 Current user: admin@MngEnvMCAP784683.onmicrosoft.com
👉🏽 Tenant ID: 93139d1e-a3c1-4d78-9ed5-878be090eba4
👉🏽 Subscription ID: dcef7009-6b94-4382-afdc-17eb160d709a


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations.

`openAIModelCapacity` is set intentionally low to `8` (8k tokens per minute) to trigger the retry logic in the load balancer (transparent to the user) as well as the priority failover from priority 1 to 2.

In [31]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"
# output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
#     f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name frc-demo-lab-private-connectivity 
👉🏽 Resource group frc-demo-lab-private-connectivity does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name frc-demo-lab-private-connectivity --location francecentral --tags source=ai-gateway 
✅ Resource group 'frc-demo-lab-private-connectivity' created ⌚ 17:10:15.620873 :3s]
{
  "id": "/subscriptions/dcef7009-6b94-4382-afdc-17eb160d709a/resourceGroups/frc-demo-lab-private-connectivity/providers/Microsoft.Resources/deployments/private-connectivity",
  "location": null,
  "name": "private-connectivity",
  "properties": {
    "correlationId": "8a846a87-9d99-40f9-bb79-2c0cffb0cc46",
    "debugSetting": null,
    "dependencies": [
      {
        "dependsOn": [
          {
            "id": "/subscriptions/dcef7009-6b94-4382-afdc-17eb160d709a/resourceGroups/frc-demo-lab-private-connectivity/providers/Microsoft.Network/networkSecurityGroups/nsg-apim",
            "resourceGroup": "frc-demo-


d:\projects\Azure-Samples\AI-Gateway\labs\private-connectivity\main.bicep(411,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is a resource type definition inaccuracy, report it using https://aka.ms/bicep-type-issues. [https://aka.ms/bicep/core-diagnostics#BCP035]




### Approve Front Door private link connection to APIM

Azure Front Door will stablsih a private link connection to the API Management service. This connection should be approved. To do that, go to the Azure portal and navigate search for `private link services` in the search bar. Then click on `Pending Connections`. You should see a connection request from Front Door to APIM. Click on it and approve the connection.

![](approve-pl-connection.png)

### Disabling APIM public network access

As per today, during the creation, the APIM service cannot disable the public network access. This is a known issue and the APIM team is working on it. As a workaround, you can disable the public network access after the deployment is completed. To do that, go to the Azure portal and navigate to your APIM service. Then click on `Networking` in the left menu. Under `Public network access`, select `Disabled` and click on `Save`. This will disable the public network access for your APIM service.

![](disable-apim-public-network-access.png)

<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [24]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    frontdoor_endpoint = utils.get_deployment_output(output, 'frontDoorEndpointHostName', 'Front Door Endpoint')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)

⚙️ Running: az deployment group show --name private-connectivity -g frc-lab-private-connectivity 
✅ Retrieved deployment: private-connectivity ⌚ 15:47:55.336378 :6s]
👉🏽 Front Door Endpoint: afd-kj24hxjjth5ho-hhgef4apefc5cvc5.b01.azurefd.net
👉🏽 APIM API Gateway URL: https://apim-fsb4hr4kq2bls.azure-api.net
👉🏽 APIM Subscription Key (masked): ****dfab


<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses. 

You will not see HTTP 429s returned as API Management's `retry` policy will select an available backend. If no backends are viable, an HTTP 503 will be returned.

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior of the backend pool.

In [ ]:
import requests, time

runs = 20
sleep_time_ms = 100
url = f"https://{frontdoor_endpoint}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}"
# url = f"{apim_resource_gateway_url}/openai/deployments/{openai_deployment_name}/chat/completions?api-version={openai_api_version}"
messages = {"messages": [
    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}
api_runs = []

# Initialize a session for connection pooling and set any default headers
session = requests.Session()
session.headers.update({'api-key': apim_subscription_key})

try:
    for i in range(runs):
        print(f"▶️ Run {i+1}/{runs}:")

        start_time = time.time()
        response = session.post(url, json = messages)
        response_time = time.time() - start_time
        print(f"⌚ {response_time:.2f} seconds")

        utils.print_response_code(response)

        if "x-ms-region" in response.headers:
            print(f"x-ms-region: \x1b[1;32m{response.headers.get("x-ms-region")}\x1b[0m") # this header is useful to determine the region of the backend that served the request
            api_runs.append((response_time, response.headers.get("x-ms-region")))

        if (response.status_code == 200):
            data = json.loads(response.text)
            print(f"Token usage: {json.dumps(dict(data.get("usage")), indent = 4)}\n")
            print(f"💬 {data.get("choices")[0].get("message").get("content")}\n")
        else:
            print(f"{response.text}\n")

        time.sleep(sleep_time_ms/1000)
finally:
    # Close the session to release the connection
    session.close()

<a id='plot'></a>
### 🔍 Analyze Load Balancing results

The priority 1 backend will be used until TPM exhaustion sets in, then distribution will occur near equally across the two priority 2 backends with 50/50 weights.  

Please note that the first request of the lab can take a bit longer and should be discounted in terms of duration.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle as pltRectangle
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = [15, 7]
df = pd.DataFrame(api_runs, columns = ['Response Time', 'Region'])
df['Run'] = range(1, len(df) + 1)

# Define a color map for each region
color_map = {'East US': 'lightpink', 'Sweden Central': 'lightyellow', 'West US': 'lightblue'}  # Add more regions and colors as needed

# Plot the dataframe with colored bars
ax = df.plot(kind = 'bar', x = 'Run', y = 'Response Time', color = [color_map.get(region, 'gray') for region in df['Region']], legend = False)

# Add legend
legend_labels = [pltRectangle((0, 0), 1, 1, color = color_map.get(region, 'gray')) for region in df['Region'].unique()]
ax.legend(legend_labels, df['Region'].unique())

plt.title('Load Balancing results')
plt.xlabel('Run #')
plt.ylabel('Response Time')
plt.xticks(rotation = 0)

average = df['Response Time'].mean()
plt.axhline(y = average, color = 'r', linestyle = '--', label = f'Average: {average:.2f}')

plt.show()

<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK

Repeat the same test using the Python SDK to ensure compatibility. Note that we do not know what region served the response; we only see that we obtained a response.

In [21]:
import time
from openai import AzureOpenAI

runs = 20
sleep_time_ms = 100

client = AzureOpenAI(
    azure_endpoint = apim_resource_gateway_url,
    api_key = apim_subscription_key,
    api_version = openai_api_version
)

for i in range(runs):
    print(f"▶️ Run {i+1}/{runs}:")

    start_time = time.time()
    raw_response = client.chat.completions.with_raw_response.create(
        model = openai_model_name,
        messages = [
            {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
            {"role": "user", "content": "Can you tell me the time, please?"}
        ])
    response_time = time.time() - start_time

    print(f"⌚ {response_time:.2f} seconds")
    print(f"x-ms-region: \x1b[1;32m{raw_response.headers.get("x-ms-region")}\x1b[0m") # this header is useful to determine the region of the backend that served the request

    response = raw_response.parse()

    if response.usage:
        print(f"Token usage:\n   Total tokens: {response.usage.total_tokens}\n   Prompt tokens: {response.usage.prompt_tokens}\n   Completion tokens: {response.usage.completion_tokens}\n")

    print(f"💬 {response.choices[0].message.content}\n")

    time.sleep(sleep_time_ms/1000)

▶️ Run 1/20:
⌚ 2.12 seconds
x-ms-region: West US
Token usage:
   Total tokens: 57
   Prompt tokens: 31
   Completion tokens: 26

💬 Oh sure, let me just check my imaginary watch. I mean, how hard can it be to look at a clock?

▶️ Run 2/20:
⌚ 1.62 seconds
x-ms-region: West US
Token usage:
   Total tokens: 81
   Prompt tokens: 31
   Completion tokens: 50

💬 Sure, I can tell you the time—when you tell me which time zone you're in, whether you want it in 12-hour or 24-hour format, and if your clock is set correctly. Otherwise, just look at your phone!

▶️ Run 3/20:
⌚ 3.87 seconds
x-ms-region: East US
Token usage:
   Total tokens: 73
   Prompt tokens: 31
   Completion tokens: 42

💬 Oh sure, let me just consult my magic crystal ball. But seriously, I can’t check the time for you. You might want to look at a clock or something. They're pretty common these days.

▶️ Run 4/20:
⌚ 1.10 seconds
x-ms-region: Sweden Central
Token usage:
   Total tokens: 67
   Prompt tokens: 31
   Completion tokens: 3

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.